# Getting test data from Elasticsearch

Here we get a fixed number of documents from Elasticsearch using a random seed and store them in JSON format.

In [19]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../..")

from heritageconnector.datastore import es, index
from elasticsearch import helpers
from tqdm.auto import tqdm
from itertools import islice
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
limit = None

# only consider docs with both a label and description field
query = {
    "query": {
        "function_score": {
            "query": {
                "bool": {
                    "must": [
                        {"exists": {"field": "graph.@rdfs:label"}},
                        {
                            "exists": {
                                "field": "data.http://www.w3.org/2001/XMLSchema#description"
                            }
                        },
                    ]
                }
            },
            "random_score": {"seed": 42, "field": "_seq_no"},
        }
    }
}

doc_generator = helpers.scan(
    client=es,
    index=index,
    query=query,
    preserve_order=True,
)

if limit:
    doc_generator = islice(doc_generator, limit)

In [21]:
# run export

output_path = f"./test_data_{limit}.jsonl"

with open(output_path, "w") as f:
    for doc in tqdm(doc_generator):
        json.dump(doc, f)
        f.write("\n")

0it [00:00, ?it/s]